## VGG16 모델 활용 이미지 식별(CIFAR10 데이터셋)

In [5]:
from keras.utils import set_random_seed
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.datasets.cifar10 import load_data

from keras.applications import VGG16

import numpy as np
import matplotlib.pyplot as plt

In [3]:
set_random_seed(11)

In [4]:
# 학습용 : 테스트용 = 8 : 2
(x_train, y_train), (x_test, y_test) = load_data()

In [8]:
# 스케일링
x_train=x_train/255.0
x_test=x_test/255.0

In [9]:
x_train.shape

(50000, 32, 32, 3)

In [28]:
# 원핫인코딩으로 변환
from sklearn.preprocessing import OneHotEncoder

# y_train
encoder=OneHotEncoder()
encoder.fit(y_train.reshape(-1,1))
y_train=encoder.transform(y_train.reshape(-1,1))
y_train=y_train.toarray()

# y_test
encoder.fit(y_test.reshape(-1,1))
y_test=encoder.transform(y_test.reshape(-1,1))
y_test=y_test.toarray()

In [20]:
# VGG16 모델

# 모델 생성(이미지넷으로 학습된 모델, 입력 형태 변경)
model_vgg=VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

# 이미지넷으로 학습된 값 그대로 사용
model_vgg.trainable = False 

model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [40]:
# 모델 생성
model=Sequential()

# vgg16모델 
model.add(model_vgg)

# 형태 변환
model.add(Flatten())

model.add(Dense(2000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))

# 출력층
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_7 (Flatten)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 1000)              513000    
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_18 (Dense)            (None, 500)               500500    
                                                                 
 dense_19 (Dense)            (None, 100)               50100     
                                                                 
 dense_20 (Dense)            (None, 10)               

In [41]:
# 모델 생성
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [43]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 검증 데이터에 대한 loss값 2번 연속 개선되지 않으면 학습 중지
stopCB=EarlyStopping(monitor='val_loss', patience=2)

In [44]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[stopCB])

Epoch 1/10
1563/1563 [==============================] - 407s 260ms/step - loss: 1.4502 - accuracy: 0.4874 - val_loss: 1.2815 - val_accuracy: 0.5469
Epoch 2/10
1563/1563 [==============================] - 398s 255ms/step - loss: 1.2855 - accuracy: 0.5474 - val_loss: 1.2223 - val_accuracy: 0.5699
Epoch 3/10
1563/1563 [==============================] - 442s 283ms/step - loss: 1.2275 - accuracy: 0.5696 - val_loss: 1.2162 - val_accuracy: 0.5809
Epoch 4/10
1563/1563 [==============================] - 469s 300ms/step - loss: 1.1884 - accuracy: 0.5844 - val_loss: 1.1762 - val_accuracy: 0.5861
Epoch 5/10
1563/1563 [==============================] - 463s 296ms/step - loss: 1.1586 - accuracy: 0.5939 - val_loss: 1.1502 - val_accuracy: 0.5947
Epoch 6/10
1563/1563 [==============================] - 446s 286ms/step - loss: 1.1302 - accuracy: 0.6049 - val_loss: 1.1336 - val_accuracy: 0.6062
Epoch 7/10
1563/1563 [==============================] - 453s 290ms/step - loss: 1.1065 - accuracy: 0.6146 - val_

In [45]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 54s 171ms/step - loss: 1.1195 - accuracy: 0.6101


[1.119463324546814, 0.6100999712944031]

In [46]:
# # 모델 저장
# from keras.models import save_model
# model061='cifar10/Model061.h5'
# save_model(model, model061)

In [47]:
# # 모델 복원
# from keras.models import load_model
# reModel=load_model(model061)

In [36]:
# 모델 학습
# reModel.fit(x_train, y_train, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 321s 205ms/step - loss: 0.5981 - accuracy: 0.7834
Epoch 2/10
1563/1563 [==============================] - 349s 223ms/step - loss: 0.5443 - accuracy: 0.8032
Epoch 3/10
1563/1563 [==============================] - 345s 221ms/step - loss: 0.4994 - accuracy: 0.8190
Epoch 4/10
1563/1563 [==============================] - 348s 223ms/step - loss: 0.4449 - accuracy: 0.8382
Epoch 5/10
1563/1563 [==============================] - 329s 211ms/step - loss: 0.4073 - accuracy: 0.8512
Epoch 6/10
1563/1563 [==============================] - 360s 230ms/step - loss: 0.3691 - accuracy: 0.8671
Epoch 7/10
1563/1563 [==============================] - 365s 233ms/step - loss: 0.3275 - accuracy: 0.8810
Epoch 8/10
1563/1563 [==============================] - 380s 243ms/step - loss: 0.3090 - accuracy: 0.8880
Epoch 9/10
1563/1563 [==============================] - 385s 246ms/step - loss: 0.2861 - accuracy: 0.8972
Epoch 10/10
1563/1563 [=======================